# yelp.com 정보 수집 크롤러

In [1]:
#1 모듈 & 라이브러리 호출
from selenium import webdriver
import selenium
import time
import os
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
import random
import math
import urllib.request
import urllib
# 엑셀에 사진 파일 첨부할 라이브러리들
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from PIL import Image
from io import BytesIO
import requests

In [2]:
#2 사용자 인터페이스 설정
print('='*100)
print('이 크롤러는 www.yelp.com의 레스토랑 이용 후기 정보를 수집합니다')
print('='*100)

key_word = input('1.크롤링할 메뉴는 무엇입니까?(예:sushi) :')
location = input('2.크롤링할 지역은 어디 입니까?(예:LA) :')
cnt = int(input('3.크롤링 할 매장수는 몇개입니까 :'))
number = int(input('4.각 매장별 크롤링할 리뷰 갯수는 몇개입니까? :'))
f_dir = input('5.파일이 저장될 경로만 쓰세요(예: /Users/kibeomkim/Desktop/folder2) :')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

f_name = s+'-'+key_word+'.txt'
fc_name = s+'-'+key_word+'.csv'
fx_name = s+'-'+key_word+'.xlsx'



이 크롤러는 www.yelp.com의 레스토랑 이용 후기 정보를 수집합니다
1.크롤링할 메뉴는 무엇입니까?(예:sushi) :burgers
2.크롤링할 지역은 어디 입니까?(예:LA) :LA
3.크롤링 할 매장수는 몇개입니까 :30
4.각 매장별 크롤링할 리뷰 갯수는 몇개입니까? :60
5.파일이 저장될 경로만 쓰세요(예: /Users/kibeomkim/Desktop/folder2) :


In [3]:
#3. 크롬드라이버 호출
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options=webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)
s_time = time.time()

In [4]:
#4. 페이지로 이동, 검색어 및 검색 조건 넣기
driver.get('https://www.yelp.com/')
driver.implicitly_wait(10)

driver.find_element_by_link_text('Restaurants').click()
time.sleep(3)
element1 = driver.find_element_by_id('search_description')
element1.send_keys(key_word)
time.sleep(0.5)
element2 = driver.find_element_by_id('search_location')
for a in range(0, 50) : 
    element2.send_keys(Keys.BACKSPACE)
element2.send_keys(location)
time.sleep(0.5)
driver.find_element_by_xpath('/html/body/yelp-react-root/div[1]/div[3]/div/div/div[1]/div[2]/div/div/div/div/div/div/form/div/div[2]/div/div[2]/button').click()
time.sleep(3)

In [5]:
#5. 전체 크롤링 시작, 반복은 총 4번
no2 = 2
no1 = 1
no3 = 1
no4 = 1
f = open(f_name, 'a', encoding='utf-8')
url_list = []
no_list = []
store_name1 = []
main_menu1 = []
stars1 = []
ratings1 = []
real_url_list1 = []
img_list1 = []

while no2 == 2 : 
    html0 = driver.page_source
    soup0 = BeautifulSoup(html0, 'html.parser')
    list0 = soup0.find_all('div', ' container__09f24__21w3G hoverable__09f24__2nTf3 margin-t3__09f24__5bM2Z margin-b3__09f24__1DQ9x padding-t3__09f24__-R_5x padding-r3__09f24__1pBFG padding-b3__09f24__1vW6j padding-l3__09f24__1yCJf border--top__09f24__1H_WE border--right__09f24__28idl border--bottom__09f24__2FjZW border--left__09f24__33iol border-color--default__09f24__R1nRO')
    for c in list0 : 
        image_src = c.find('img', ' photo-box-img__09f24__3F3c5')['src']
        img_list1.append(image_src)

        print('%s 번째 그림 다운로드 했습니다' %no1)
        f.write('%s 번째 그림 다운로드 했습니다' %no1 + '\n')
        print('%s 번째 게시글 정보를 수집합니다' %no1)
        f.write('%s 번째 게시글 정보를 수집합니다' %no1 + '\n')
        print('-'*100)
        f.write('-'*100)
        
        print('1.판매순위: %s' %no1)
        f.write('1.판매순위: %s' %no1 + '\n')
        no_list.append(no1) 
        
        store_name = c.find('a', ' link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95').get_text().replace('\n', "").strip()
        print('2.매장명 : %s' %store_name)
        f.write('2.매장명 : %s' %store_name+'\n')
        url = c.find('a',' link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95' )['href']
        url_list.append(url)
        store_name1.append(store_name)
        
        main_menu = c.find('div', ' priceCategory__09f24__2IbAM display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO').find('div', class_=' border-color--default__09f24__R1nRO').get_text().replace('/n', "").strip()
        print('3.주메뉴 : %s' %main_menu)
        f.write('3.주메뉴 : %s' %main_menu+'\n')
        main_menu1.append(main_menu)
        
        stars = c.find('span', class_=' display--inline__09f24__3iACj border-color--default__09f24__R1nRO').find('div')['aria-label']
        print('4.평점 : %s' %stars)
        f.write('4.평점 : %s' %stars+'\n')
        stars1.append(stars)
        
        rating_number = c.find('span', class_=' text__09f24__2tZKC reviewCount__09f24__EUXPN text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0').get_text().replace('\n', "").strip()
        print('5.상품평 수 : %s' %rating_number)
        f.write('5.상품평 수 : %s' %rating_number+'\n')
        ratings1.append(rating_number)
        
        store_url1 = 'https://www.yelp.com/'+url
        real_url_list1.append(store_url1)
        print('6.%s번째 매장 URL :' %no1,store_url1)
        f.write('6.%s번째 매장 URL :' %no1+store_url1+'\n')
        
        print('='*100)
        print('\n')
        f.write('='*100+'\n')
        f.write('\n')
        
        no1 += 1
        no3 += 1
        if no1 > cnt : 
            break
    print('%s 번째 페이지까지 %s 건 정보 수집 완료===============' %(no4, (no3-1)))
    print('\n')
    f.write('%s 번째 페이지까지 %s 건 정보 수집 완료===============' %(no4, (no3-1))+'\n')
    f.write('\n')
    if no1 > cnt : 
        break
    try : 
        driver.find_element_by_link_text('%s' %(no4+1)).click()
        time.sleep(5)
    except : 
        break
    else : 
        no4 +=1 
        no3 = 1
driver.quit()
f.close()





1 번째 그림 다운로드 했습니다
1 번째 게시글 정보를 수집합니다
----------------------------------------------------------------------------------------------------
1.판매순위: 1
2.매장명 : Burger Lounge
3.주메뉴 : Burgers
4.평점 : 4 star rating
5.상품평 수 : 614
6.1번째 매장 URL : https://www.yelp.com//adredir?ad_business_id=9HTgwGhDDtGyx-QOBYcuLg&campaign_id=K1p3kKv0xYbmb6Qu5SpHPA&click_origin=search_results&placement=above_search&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fburger-lounge-santa-monica-3&request_id=7f13c6c949571b9e&signature=abb78a0d8a18287a45bcb736755e0f81d2d2c1a4d6267efc88bdb2075ea37bb2&slot=0


2 번째 그림 다운로드 했습니다
2 번째 게시글 정보를 수집합니다
----------------------------------------------------------------------------------------------------
1.판매순위: 2
2.매장명 : Honeybee Burger
3.주메뉴 : Burgers, Fast Food
4.평점 : 4.5 star rating
5.상품평 수 : 367
6.2번째 매장 URL : https://www.yelp.com//adredir?ad_business_id=o0oFDbQg2Wc-lE--GY7-Nw&campaign_id=Zm97-b0guTwuuBZQIK8Oqg&click_origin=search_results&placement=above_search

27 번째 그림 다운로드 했습니다
27 번째 게시글 정보를 수집합니다
----------------------------------------------------------------------------------------------------
1.판매순위: 27
2.매장명 : BurgerShop
3.주메뉴 : American (Traditional), Fast Food, Burgers
4.평점 : 4.5 star rating
5.상품평 수 : 77
6.27번째 매장 URL : https://www.yelp.com//adredir?ad_business_id=Nm1iV8Aaat4EF_0hPevXhg&campaign_id=k5toZhxH32zTMokRHw5FWA&click_origin=search_results&placement=above_search&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fburgershop-los-angeles-2&request_id=cfc5e6ec83ca7305&signature=17c56aebe42edd26200af8d39a31655c65dc0f17f2d6a410d70278dcfb042f38&slot=0


28 번째 그림 다운로드 했습니다
28 번째 게시글 정보를 수집합니다
----------------------------------------------------------------------------------------------------
1.판매순위: 28
2.매장명 : Burger Lounge
3.주메뉴 : Burgers
4.평점 : 4 star rating
5.상품평 수 : 614
6.28번째 매장 URL : https://www.yelp.com//adredir?ad_business_id=9HTgwGhDDtGyx-QOBYcuLg&campaign_id=K1p3kKv0xYbmb6Qu5SpHPA&click_origin=search_results&

In [6]:
e_time = time.time()
total_time = e_time - s_time

store_list23 = pd.DataFrame()
store_list23['판매순위'] = no_list
store_list23['매장명'] = store_name1
store_list23['주메뉴'] = main_menu1
store_list23['상품평점'] = stars1
store_list23['상품평수'] = ratings1
store_list23['URL주소'] = real_url_list1

store_list23.to_csv(fc_name, encoding='utf-8-sig')
store_list23.to_excel(fx_name)


#엑셀에 사진 넣기 
wb = openpyxl.load_workbook(fx_name)
ws = wb.worksheets[0]

for a in range(2, no1+1) : 
    ws.row_dimensions[a].height =  200
    
ws.column_dimensions['C'].width = 50


nor = 2
for a in img_list1 : 
    res = requests.get(a)
    image_data = BytesIO(res.content)
    img = openpyxl.drawing.image.Image(image_data)
    ws.add_image(img, anchor='C%s' %nor)
    nor +=1
wb.save(fx_name)
print('엑셀에 사진 첨부 완료')

print('='*100)
print('전체 매장 목록을 수집했습니다')
print('총 소요시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)

엑셀에 사진 첨부 완료
전체 매장 목록을 수집했습니다
총 소요시간은 29.5 초 입니다


In [7]:
no5 = 0
no6 = 1
no7 = 1
no8 = 1
no13 = 1
no15 = 1

print('='*100)
print('지금부터 각 매장별 리뷰 수집을 시작합니다')
print('리뷰 내용이 많아서 다소 시간이 소요될 수 있으니 잠시만 기다려 주세요~~')
print('리뷰 수집을 요청하신 매장 수는 총 %s 개 이고 각 매장별로 %s 건의 리뷰 수집을 요청하겠습니다~~' %(cnt, number))
print('='*100)
print('\n')
print('\n')
print('수집을 요청하신 리뷰 %s 건의 수집을 시작합니다~^^' %number)
print('현재 총 %s개의 매장 중 %s번째 매장인 %s 매장의 리뷰를 수집중입니다~~~' %(cnt, no6, store_name1[no5]))
print('참고로 이 매장의 총 리뷰 갯수는 현재 %s 개 입니다~' %ratings1[no5])
print('\n')



for e in real_url_list1 : 
    user_name1 = []
    located_area1 = []
    stars1 = []
    reviews1 = []
    cons1 = []
    
    no6 +=1
    path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
    options=webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
    driver = webdriver.Chrome(path, options=options)
    
    driver.get(e)
    time.sleep(5)
    try : 
        os.makedirs('/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'+s+'-'+key_word+'/'+'%s' %(store_name1[no5]))
        os.chdir('/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'+s+'-'+key_word+'/'+'%s' %(store_name1[no5]))
    except FileExistsError : 
        no5 +=1
        driver.quit()
        continue

    f = open('%s review.txt' %store_name1[no5], 'a', encoding='utf-8')

    
    no9 = 1
    while no9 == 1 : 
        html1 = driver.page_source
        soup1 = BeautifulSoup(html1, 'html.parser')
        list112 = soup1.find_all('div', ' review__373c0__13kpL border-color--default__373c0__3-ifU')
        time.sleep(3)
        
        for q in list112 : 
            print('%s 번째 매장의 %s 번째 리뷰 정보를 수집합니다==================' %(no7, no8))
            f.write('%s 번째 리뷰 정보를 수집합니다=================' %no8+'\n')
            user_name = q.find('div', ' user-passport-info border-color--default__373c0__3-ifU').find('a', ' link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE').get_text().replace('\n', "").strip()
            user_name1.append(user_name)
            print('1.리뷰작성자명: %s' %user_name)
            f.write('1.리뷰작성자명: %s' %user_name+'\n')
            
            try : 
                located_area = q.find('div', ' responsive-hidden-small__373c0__2vDff border-color--default__373c0__3-ifU').find('span', ' text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-').get_text().replace('\n', "").strip()
                located_area1.append(located_area)
                print('2.리뷰작성지역: %s' %located_area)
                f.write('2.리뷰작성지역: %s' %located_area+'\n')
            except : 
                print('2.리뷰작성지역: 입력된 정보 없음')
                f.write('2.리뷰작성지역: 입력된 정보 없음'+'\n')
        
            stars = q.find('div', ' margin-t1__373c0__oLmO6 margin-b1-5__373c0__2Wblx border-color--default__373c0__3-ifU').find('span').find('div')['aria-label']
            stars1.append(stars)
            print('3.리뷰평점: %s' %stars)
            f.write('3.리뷰평점: %s' %stars+'\n')
        
            reviews_date = q.find('div', ' margin-t1__373c0__oLmO6 margin-b1-5__373c0__2Wblx border-color--default__373c0__3-ifU').find('span', ' text__373c0__2Kxyz text-color--mid__373c0__jCeOG text-align--left__373c0__2XGa-').get_text().replace('\n', "").strip()
            reviews1.append(reviews_date)
            print('4.리뷰작성일자: %s' %reviews_date)
            f.write('4.리뷰작성일자: %s' %reviews_date+'\n')
        
            review_contents = q.find('p', ' text__373c0__2Kxyz comment__373c0__1M-px text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-').get_text().replace('\n', "").strip()
            cons1.append(review_contents)
            print('5.리뷰내용: %s' %review_contents)
            f.write('5.리뷰내용 : %s' %review_contents+'\n')
            print('\n')
            f.write('\n')
            
            no13 +=1 
            no8 +=1 
            if no13 > number : 
                break
        no15 += 1
        if no13 > number : 
            break
        for y in range(0, 31) : 
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        time.sleep(3)
        try : 
            driver.find_element_by_link_text('%s' %no15).click()
            time.sleep(5)
        except : 
            try : 
                driver.find_element_by_link_text('%s' %no15).click()
                time.sleep(3)
            except : 
                try :
                    driver.find_element_by_link_text('%s' %no15).click()
                    time.sleep(3)
                except : 
                    break
    no7 += 1 
    no5 += 1
    no8 = 1
    no13 = 1
    no15 = 1
    time.sleep(3)
    f.close()
    driver.quit()
    
    review_file = pd.DataFrame()
    review_file['리뷰작성자명'] = user_name1
    review_file['리뷰작성지역'] = located_area1
    review_file['리뷰평점'] = stars1
    review_file['리뷰작성일자'] = reviews1
    review_file['리뷰내용'] = cons1
    
    review_file.to_csv('%s review.csv' %store_name1[no5-1], encoding='utf-8-sig')
    review_file.to_excel('%s review.xlsx' %store_name1[no5-1])

    

지금부터 각 매장별 리뷰 수집을 시작합니다
리뷰 내용이 많아서 다소 시간이 소요될 수 있으니 잠시만 기다려 주세요~~
리뷰 수집을 요청하신 매장 수는 총 30 개 이고 각 매장별로 60 건의 리뷰 수집을 요청하겠습니다~~




수집을 요청하신 리뷰 60 건의 수집을 시작합니다~^^
현재 총 30개의 매장 중 1번째 매장인 Burger Lounge 매장의 리뷰를 수집중입니다~~~
참고로 이 매장의 총 리뷰 갯수는 현재 614 개 입니다~


1 번째 매장의 1 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Crystal Y.
2.리뷰작성지역: San Francisco, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 12/21/2020
5.리뷰내용: Note: Open during the quarantine. Here is a hot tip since outdoor dining was banned by Governor Newsom recently: there is a Promenade plaza just one block away. Next to the fountains are ledges one can sit on and eat takeout on. We did just that so we could have the food right away while it was still warm. Modern burger joint with both healthy and traditional options. Good place to bring people as it will accommodate vegan/gluten free/keto lifestyles. I decided on the quinoa vegan burger with vegan cheese and caramelized onions. Another person in my pod had a turkey burger. Another had a grilled cod

1 번째 매장의 21 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Tiffany D.
2.리뷰작성지역: West Los Angeles, Los Angeles, CA
3.리뷰평점: 4 star rating
4.리뷰작성일자: 5/9/2018
5.리뷰내용: A coworker invited me to lunch today and said she specifically craved a quinoa burger so we decided to eat here for lunch. I came with low expectations since I haven't had a burger in a long time. To my surprise though, their quinoa burger is actually quite delicious! What defines a good veggie burger to me is its juiciness without being drenched in condiments. Burger lounge got it just right! They grilled the patty until it was crisp on the edges & they pack it with small bits of vegetables evenly through the party. The white cheese was a bit much so I'll pass on that next time but keep the sauces since it wasn't overpowering.Although, I was debating ordering fries or onion rings. I decided to pass and am quite proud of my decision since the quinoa burger by itself was quite filling! Not too shabby for $8. I give this place 4 s

1 번째 매장의 41 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Tessa C.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 6/30/2019
5.리뷰내용: Came here for the first time today. Wow!! What an amazing find! I love love love burgers and this place went right to the top of my favorites list! The workers were so nice and helpful. I love the outside patio they have! That was a treat :) And the food... SO yummy!!! It was so fresh and delicious. I can't wait to go back. I wish they had these everywhere!


1 번째 매장의 42 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Claudia C.
2.리뷰작성지역: Torrance, CA
3.리뷰평점: 4 star rating
4.리뷰작성일자: 12/13/2017
5.리뷰내용: Came here for a quick lunch with my coworkers. I love that the burger tasted fresh and grass-fed! I can't stop thinking about the onion rings here. So crunchy, crisp, and delicious. The breading isn't like other onion ring breading--it's much more smooth and almost artisan style.I'll definitely be back for the onion rings because it's been months since I

2 번째 매장의 1 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Ingrid S.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 12/26/2020
5.리뷰내용: Woke up with a donut craving this morning, so I drove downtown to Dōnatsu for some vegan donuts. While on my way back to the Valley (yes, I drove all the way downtown from the valley for some donuts. Don't judge me.)  I realized that I should probably eat a meal that wasn't just sugar. I looked up "vegan food" on Yelp and Honeybee Burger came up. I pulled up to a cute little building that I thought was closed, until I noticed the sign to go upstairs. I'm sure the place looks different regularly, but due to Covid you could not enter the building and the patio seating was all covered.Near the front door was a small window with a doorbell and a sign that indicated you needed to ring the bell for service. Shortly a woman appeared. I ordered a honeybee combo which includes a burger, fries, and shake. You have your choice of either an impossible patty 

2 번째 매장의 21 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Amanda L.
2.리뷰작성지역: Baldwin Hills, Baldwin Hills, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 12/18/2020
5.리뷰내용: Honeybee Box with the Impossible Patty is to due for and the Vegan Ranch is amazing when paired with the burger and the "frots" (fries and tots). I love the atmosphere of this place and the food is amazing. EVEN THE NO MILK STRAWBERRY MILKSHAKE WAS INCREDIBLE!!!


2 번째 매장의 22 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Nela V.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 2 star rating
4.리뷰작성일자: 7/20/2020
5.리뷰내용: This is probably my least favorite vegan burger spot in LA. I ordered when they were having a buy one, get one free special so I ordered 2 sliders with Impossible meat and some sweet potato frots (fries & tater tots). Referring to the menu...the sliders were nothing like the images. The sliders were kind of sad-looking, bland and there was just nothing special about them. The frots were room temperature, not fresh and soggy. I orde

...

23 번째 매장의 21 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Tony Z.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 7/21/2020
5.리뷰내용: Called and let them know they didn't have combo options when ordering delivery so I got a drink and nice note


23 번째 매장의 22 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Lorena S.
2.리뷰작성지역: Tarzana, Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 7/17/2019
5.리뷰내용: Life hack: Order on GrubHub write in all of your requests, prepay and have it delivered or pick up. We opted to walk over and pick it up. It is a neighborhood spot, so be sure to smile or nod and return the greetings you are likely to receive. At the window our box was ready! I started to ask about the sauces and he said they were all there. It was perfect, everything was as requested. I have to say the Onion Rings are a must. Also shared fresh cut french fries, my grilled cheese with added tomatoe was tasty and for the omnivore a pastrami & burger (pastrami burger?) I'm a vegetarian folk

23 번째 매장의 41 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Yvonne F.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 7/13/2019
5.리뷰내용: Let me start this off by saying that I'm not a burger fan. Even what people tell me are the best burgers I find to be just okay. I rarely finish them and end up donating my leftovers to my husband, who loves any burger in any way, shape, or form.But George's is good. If anything, go for the onion rings. And then still get a burger.I got the pastrami burger and ate the entire thing in one sitting. The pastrami is tasty and not overpowering, there's just the right amount of sauce and fresh lettuce/onion, and the cheddar! These guys use the real deal, longhorn cheddar and it's what makes the burger. And they don't microwave it on, it has the crunchy cheddar bits from being on the griddle and oh my is it delish. And the onion rings!!!I ordered take out and everything was thoughtfully and well packaged. There was sauce for the onion rings and peppers

24 번째 매장의 1 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Christian T.
2.리뷰작성지역: Glendale, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 10/4/2020
5.리뷰내용: This review is pretty long overdue as I've been coming to Uncle Johnny's for the last two years. Quite simply one of the best burgers you will find in Los Angeles. I remember my first bite into the beautiful smash burger patty fried up to perfection, the soft bun, and the incredible special sauce. My taste buds died and went to heaven. Now the owner has added a spicy chicken sandwich to the menu and it's arguably even better than the burger. Not too hot but moist and bursting with flavor, topped off by the same secret sauce from the burger.During COVID they are now set-up outside Glendale Tap usually Friday/Saturday and sometimes Thursdays. Make sure to go early if you want to snag a chicken sandwich as they can sell out quickly!Also just wanted to add that they owner, Dwayne, and his son are super nice and welcoming! They definitely make you fee

24 번째 매장의 21 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Hoon C.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 2/22/2020
5.리뷰내용: Add Uncle Johnny's to the list of elite burgers in our city:  Hinano Bar in Venice, In-n-out everywhere, Burgers Never Say Die (BNSD) in Silverlake, and now Uncle Johnny's in the downtown Arts District.The burger is a 1/4 pound smashed style burger.  There's a nice charred crust like BNSD but the patty is thicker.  The patty has the right amount of fat (meaning a lot - which is why, in my view, HiHo is not an elite burger).  They seem to melt and char and incorporate a bit of cheese into the char on one side of the patty and also have a slice that is allowed to melt more  traditionally.  They top the patty with plenty of pickles and onions and their version of thousand island dressing to add just the right punch of acid.  I opted to add lettuce and tomato (classic style) but you can easily forego if you really want a pure, carnivore experience (or

24 번째 매장의 41 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: William O.
2.리뷰작성지역: Central LA, Los Angeles, CA
3.리뷰평점: 5 star rating
4.리뷰작성일자: 6/23/2019
5.리뷰내용: Found the tent in the patio of Glendale Tap bar (on San Fernando in Glendale).Incredible burger. Nothing fancy about it. No trendy toppings. No gimmicky advertising. Just honest and delicious food.Imagine the best possible burger at a family picnic. This is it. No fries or true sides at this point, but there's talk of adding it. Fingers crossed.


24 번째 매장의 42 번째 리뷰 정보를 수집합니다==================
1.리뷰작성자명: Vincent B.
2.리뷰작성지역: Stow, OH
3.리뷰평점: 5 star rating
4.리뷰작성일자: 5/25/2018
5.리뷰내용: Where do I start ? Well first and foremost if you don't know about BBQ just stop by this spot you will learn everything you need to know , this is hands down some of the best BBQ I have had in a very long time. The thing that makes this BBQ stand out is the attention to detail, you can tell with every bite that this food was prepared by someone who loves 